<a href="https://colab.research.google.com/github/JayeonKangNature/2022_Summer_Internship/blob/main/Self_Study_Code/Week3_Process_from_configuring_to_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size = 10>**Week 3**

<font size = 4> Jayeon Kang

---



I run my code before the usage of ObjectDetector calss [line 29] because all codes after line 29 do not run correctely due to not use Detector information.... <br>
but I write code and how does code run.

# <font size = 5> Create environment

In [ ]:
import torch
import os

# Configuring the prerequisites for Object Detection: config.py in PyImagesearch

## Define Path

In [ ]:
# Mounting
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Base path
Base_Path = '/content/drive/MyDrive/RC4_2022_summer_internship/Traffic_Signs'

# Define our dataset
# input images and annotation CSV files
Annots_Path = os.path.sep.join([Base_Path,'train/_annotations.csv'])
Images_Path = os.path.sep.join([Base_Path,'train'])
# output path to base output directory
Base_Output = Base_Path +'/output'

# Create seperate for detector and label encoder
# Model Path
Model_Path = os.path.sep.join([Base_Output,'detector.pth'])
Le_Path = os.path.sep.join([Base_Output,'le.picle'])
Plot_Path = os.path.sep.join([Base_Output,'plots'])
Test_Path = os.path.sep.join([Base_Output,'test_paths.txt'])

Mounted at /content/drive


In [ ]:
print(Images_Path)
print(Annots_Path)
print(Model_Path)

/content/drive/MyDrive/RC4_2022_summer_internship/Traffic_Signs/train
/content/drive/MyDrive/RC4_2022_summer_internship/Traffic_Signs/train/_annotations.csv
/content/drive/MyDrive/RC4_2022_summer_internship/Traffic_Signs/output/detector.pth


## Train an Object Detector from scratch in PyTorch

In [ ]:
# Element in Script of Config

# Determine the current device based on that set the pin memory flag
# Due to complex computation, use GPU (CUDA-compatible GPU) insted of CPU
# CUDA provide GPU as CUDA-compatible GPU
Device = 'cuda' if torch.cuda.is_available() else 'cpu'
Pin_memory = True if Device == 'cuda' else False

# Specify Mean and Standard Deviation for ImageNet to use PyTorch's transforms during data preparation
Mean = [0.485, 0.456, 0.406] # compoment of mean value: [mean of channel-wise, mean of width-wise, mean of height-wise]
STD = [0.229, 0.224, 0.225] # compoment of STD value: [STD of channel-wise, STD of width-wise, STD of height-wise]

# Initialization of hyperparameters (learning rate, number of epch and batch size):
Init_LR = 1e-4 # learningrate
Num_epoch = 20 # number of epoch
Batch_size = 32 # Batch size

# Specify the loss weights (the ground truth - the predicted output of the model)
labels = 1.0
BBox = 1.0

**Pin Memory <br>**
Pinned memory is virtual memory pages that are specially marked so that they cannot be paged out. They are allocated with special system API function calls. The important point for us is that CPU memory that serves as the source of destination of a DMA transfer must be allocated as pinned memory.<br>
**Application Programming Interface (API)**<br>
A set of definitions and protocols to build and integrate application software<br>
**CUDA**<br>
Parallel computing platform and programming model that enables dramatic increases in computing performance by harnessing the power of the graphics processing unit (GPU).<br>
**CPU and GPU**<br>


# Create the custom Object Detetion Data processor: CustomTesorDataset.py in PyImagesearch

In [ ]:
# __init__.py script
# import the necessary packages
from torch.utils.data import Dataset

class CustomTensorDataset(Dataset):
  def __init__(self, tensors, transforms=None):
    self.tensors = tensors
    self.transforms = transforms

  def __getitem__(self, index):
    # grab the image, label, bounding box coordinates
    image = self.tensors[0][index]
    label = self.tensors[1][index]
    bbox = self.tensors[2][index]

    # transpose the image such that its channel dimension becomes the leading one
    image = image.permute(2, 0, 1)
    # apply transforms if it is in
    if self.transforms:
      image = self.transforms(image)

    return (image, label, bbox)

  def __len__(self):
      # return the size of the dataset
      return self.tensors[0].size(0)

# Building the Object Detection Architecture: bbox_regressor.py script

In [ ]:
from torch.nn import Dropout
from torch.nn import Identity
from torch.nn import Linear
from torch.nn import Module
from torch.nn import ReLU
from torch.nn import Sequential
from torch.nn import Sigmoid

In [ ]:
class ObjectDetector(Module):
  def __init__(self, basemodel, numClasses):
    super(ObjectDetector, self).__init__()
    self.basemodel = basemodel
    self.numClasses = numClasses

    # build the regressor head for outputting the bouding box cordiates
    self.regressor = Sequential(
        Linear(basemodel.fc.in_features, 128),
        ReLU(),
        Linear(128, 64),
        ReLU(),
        Linear(64, 32),
        ReLU(),
        Linear(32, 4),
        Sigmoid()
    )


    self.classifier = Sequential(
        Linear(basemodel.fc.in_features, 512).
        ReLU(),
        Dropout(),
        Linear(512, 512),
        ReLU(),
        Dropout(),
        Linear(512, self.numClasses)
    )

    self.baseModel.fc = Identity()

  def forward(self, x):
      features = self.baseModel(x)
      bboxes = self.regressor(features)
      classLogits = self.classifier(features)
      return (bboxes, classLogits)


**Bounding box**<br>
The bounding box is predicted with a loss function that gives the error between the predicted and ground truth bounding box. Its purpose is to reduce the range of search for the object features and thereby conserve computing resources.

# Training the Object Detection Model

In [ ]:
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.optim import Adam
from torchvision.models import resnet50
from sklearn.model_selection import train_test_split
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import time
import cv2
import os

In [ ]:
data = []
labels = []
bboxes = []
imagePaths = []

In [ ]:
rows = open(Annots_Path).read().strip().split('\n') # load csv file as form of list

In [ ]:
print(rows[1:10]) # there is empty value,
rows = list(filter(None, rows)) # remove empty value by using filter
print(rows) # result but we do not need imformation about columns as processing
head_information = rows[0]
rows = rows[1:]
print(rows) # result
print(head_information)

['1655738848713---0-0---0-0_jpg.rf.84671b1fac0bd4d4d6c6f66262bcfb45.jpg,416,416,Stop,267,13,296,46', '', '1657282765453--0-0--0-0_jpg.rf.84d20d12e82ab09a0f751532f5da424d.jpg,416,416,Stop,192,198,240,259', '', '', '1655739273890---0-351549---0-0_jpg.rf.84ab18e1bf94a98c389472d6ff3bd736.jpg,416,416,Turn left,279,44,297,78', '', '', '']
['filename,width,height,class,xmin,ymin,xmax,ymax', '1655738848713---0-0---0-0_jpg.rf.84671b1fac0bd4d4d6c6f66262bcfb45.jpg,416,416,Stop,267,13,296,46', '1657282765453--0-0--0-0_jpg.rf.84d20d12e82ab09a0f751532f5da424d.jpg,416,416,Stop,192,198,240,259', '1655739273890---0-351549---0-0_jpg.rf.84ab18e1bf94a98c389472d6ff3bd736.jpg,416,416,Turn left,279,44,297,78', '1657282741579--0-0--0-0_jpg.rf.84682a05c739578aacaf9c9ef5e9dcd5.jpg,416,416,Turn left,93,186,207,338', '1657282681637--0-0--0-0_jpg.rf.843c63e6816f498b61d321d31a6a6aac.jpg,416,416,Turn right,186,106,288,239', '1655993661946---0-0--0-1_jpg.rf.84c1d92a458d2699d2fc2e302e38ecb1.jpg,416,416,Achtung,200,46,

In [ ]:
# Because there are so many many many images in our file, a lot of runtime is required.
# So, I treat 400 images
rows = rows[:400]
for row in rows:
  # take out row elements into list as filename, bouding box, class label
  row = row.split(',')
  filename = row[0]
  startX = row[4]
  startY = row[5]
  endX = row[6]
  endY = row[7]
  label = row[3]

    #After splitting the row values into a tuple of individual values,
    # we first single out the image path
  imagePath = os.path.sep.join([Images_Path,filename]) # this code return as A/B/C if input os.path.sep.join([A,B,C])
  image = cv2.imread(imagePath)
  (h, w) = image.shape[:2]

  # scaling the bounding box coordinates process
  # relative to the spatial dimensions (h, w) of the input image
  # to make them values in the range from 0 to 1 by using values of h and w
  startX = float(startX)/w
  startY = float(startY)/h
  endX = float(endX)/w
  endY = float(endY)/h

  image = cv2.imread(imagePath) # load the image
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert color space
  image = cv2.resize(image, (224, 224)) # resizing

  # update lists with unpacked values
  data.append(image)
  labels.append(label)
  bboxes.append([startX, startY, endX, endY])
  imagePaths.append(imagePath)


In [ ]:
print(bboxes[0])
print(labels)
print(imagePaths[0])

[0.6418269230769231, 0.03125, 0.7115384615384616, 0.11057692307692307]
['Stop', 'Stop', 'Turn left', 'Turn left', 'Turn right', 'Achtung', 'Speed limit -30-', 'Speed limit -30-', 'Stop', 'Stop', 'Turn left', 'Stop', 'Turn left', 'Turn left', 'Turn left', 'Speed limit -30-', 'Turn right', 'Stop', 'Turn left', 'Achtung', 'Speed limit -30-', 'Turn left', 'Achtung', 'Stop', 'Stop', 'Achtung', 'Achtung', 'Achtung', 'Speed limit -30-', 'Turn left', 'Turn right', 'Stop', 'Turn left', 'Aufhebung', 'Achtung', 'Turn left', 'Turn left', 'Turn left', 'Aufhebung', 'Stop', 'Stop', 'Turn right', 'Achtung', 'Stop', 'Turn right', 'Turn left', 'Stop', 'Stop', 'Turn right', 'Turn right', 'Turn left', 'Turn left', 'Turn right', 'Stop', 'Turn left', 'Turn left', 'Stop', 'Stop', 'Speed limit -30-', 'Turn right', 'Speed limit -30-', 'Achtung', 'Turn left', 'Turn left', 'Turn left', 'Turn right', 'Turn left', 'Speed limit -30-', 'Turn left', 'Stop', 'Stop', 'Turn left', 'Speed limit -30-', 'Speed limit -30-',

In [ ]:
print(data[0])

[[[ 67  43  63]
  [ 64  43  65]
  [ 27  15  41]
  ...
  [ 51  25  36]
  [ 43  17  28]
  [ 44  18  29]]

 [[ 67  44  64]
  [ 56  36  58]
  [ 19   5  32]
  ...
  [ 53  27  38]
  [ 46  20  31]
  [ 44  18  29]]

 [[ 62  39  61]
  [ 34  14  37]
  [ 20   4  30]
  ...
  [ 55  29  40]
  [ 51  25  36]
  [ 46  20  31]]

 ...

 [[107  76  89]
  [105  73  86]
  [115  83  96]
  ...
  [ 50  20  30]
  [ 83  53  63]
  [ 80  50  60]]

 [[102  70  84]
  [ 87  55  68]
  [110  78  91]
  ...
  [ 47  17  27]
  [ 77  47  57]
  [ 79  49  59]]

 [[106  74  87]
  [ 90  58  71]
  [102  70  83]
  ...
  [ 43  13  23]
  [ 67  37  47]
  [ 78  48  58]]]


**os.path.sep.join()**<br>
The return value is the concatenation of path and any members of paths with exactly one directory separator (os. sep) following each non-empty part except the last, meaning that the result will only end in a separator if the last part is empty<br>
**Bounding Box**<br>
A bounding box is an imaginary rectangle that serves as a point of reference for object detection and creates a collision box for that object

In [ ]:
# Data preprocessing step
# To spped up processing data,
# convert the data, class labels, bounding boxes, and image paths to NumPy array
data = np.array(data, dtype="float32")
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
imagePaths = np.array(imagePaths)

In [ ]:
# perform label encoding on the labels
# process to map characters into numbers (numericization)
le = LabelEncoder() # Using scikit-learn's LabelEncoder(),
labels = le.fit_transform(labels) # convert sting format of labels in list
                                  # into respective indices

# Split data process (test and training)by train_test_split()
# As assigning test size = 0.2, 20% of datas are used as test data
# and the remaining 20% for testing
split = train_test_split(data, labels, bboxes, imagePaths,test_size=0.20, random_state=42)

# Unpack the data split
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

# Convert NumPy arrays to PyTorch tensors
(trainImages, testImages) = torch.tensor(trainImages), torch.tensor(testImages)
(trainLabels, testLabels) = torch.tensor(trainLabels), torch.tensor(testLabels)
(trainBBoxes, testBBoxes) = torch.tensor(trainBBoxes), torch.tensor(testBBoxes)

In [ ]:
# Define normalization transforms
transforms = transforms.Compose([
	transforms.ToPILImage(),
	transforms.ToTensor(),
	transforms.Normalize(mean=Mean, std=STD)
])

# convert NumPy arrays to PyTorch datasets(DS)
trainDS = CustomTensorDataset((trainImages, trainLabels, trainBBoxes),
	transforms=transforms)
testDS = CustomTensorDataset((testImages, testLabels, testBBoxes),
	transforms=transforms)

# calculate steps per epoch for training and validation set
trainSteps = len(trainDS) // Batch_size
valSteps = len(testDS) // Batch_size

# create data loaders
trainLoader = DataLoader(trainDS, batch_size=Batch_size,
	shuffle=True, num_workers=os.cpu_count(), pin_memory=Pin_memory)
testLoader = DataLoader(testDS, batch_size=Batch_size,
	num_workers=os.cpu_count(), pin_memory=Pin_memory)

In [ ]:
# Write the testing image paths to disk to use then
# when evaluating/testing our object detector
f = open(Test_Path, "w")
f.write("\n".join(testPaths))
f.close()

# load the ResNet50 network
resnet = resnet50(pretrained=True)

# freeze all ResNet50 layers (the weight) for them not to be updated
# during the training process
# The reason why do freeze is
# that the weights of base model will be left untouched
for param in resnet.parameters():
	param.requires_grad = False
  # create our custom object detector model and flash it to the current

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
# Create our custome model instance and load it to the current device step
# with the model prerequisite
objectDetector = ObjectDetector(resnet, len(le.classes_))
objectDetector = objectDetector.to(Device)

# define our loss functions
classLossFunc = CrossEntropyLoss()
bboxLossFunc = MSELoss() # MSELoss: Mean Squared error loss

# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(objectDetector.parameters(), lr=Init_LR)

# initialize a dictionary
# to store training history and track the training loss and other metrics
H = {"total_train_loss": [], "total_val_loss": [], "train_class_acc": [],
	 "val_class_acc": []}

AttributeError: ignored

In [ ]:
# loop over epochs
startTime = time.time() # to note start time
for e in tqdm(range(Num_epoch)): # trainning as much as epoch
	# set the model in training mode
	objectDetector.train()
	# initialize the total loss of training and validation
	totalTrainLoss = 0
	totalValLoss = 0

	# initialize the number of correct predictions in the training and validation
	trainCorrect = 0
	valCorrect = 0

	for (images, labels, bboxes) in trainLoader: # take out element
																		# as taking out one by one of set
		(images, labels, bboxes) = (images.to(Device),\
		                            labels.to(Device),\
		                            bboxes.to(Device))
		# perform a forward pass
		Predictions = objectDetector(images) # perform model about given image
				# and return output (prediction) as label and bounding box in the order
		bboxLoss = bboxLossFunc(Predictions[0], bboxes)
		classLoss = classLossFunc(Predictions[1], labels)
		# calculate training loss
		totalLoss = (bboxes * bboxLoss) + (labels * classLoss)

		opt.zero_grad() # zero out the gradient not to be tracking any unnecessary
										# information when we train our neural network
		totalLoss.backward() # perform backwardpropagation
		opt.step() # uptate weight

		totalTrainLoss += totalLoss
		trainCorrect += (Predictions[1].argmax(1) == labels).type(
				torch.float).sum().item()

		# NOTE: 'Reset the gredient. by using opt.zero_grad() every epoch'
		# we need gradient pertaining to the current step.
		# So, by fact that backward function keeps accumulating the gradient altogether,
		# we have to reset gredient every epoch

		with torch.no_grad(): # switch off automatic gradients (.no_grad())
			objectDetector.eval()
			for (images, labels, bboxes) in testLoader:
				(images, labels, bboxes) = (images.to(Device), labels.to(Device), bboxes.to(Device))
				Predictions = objectDetector(images)
				bboxLoss = bboxLossFunc(Predictions[0], bboxes)
				totalLoss = classLossFunc(Predictions[1])
				totalValLoss += totalLoss
				valCorrect += (Predictions[1].argmax[1]==labels).type(torch.float).sum().item()

			# calculate the average training and validation loss
			avgTrainLoss = totalTrainLoss / trainSteps
			avgValLoss = totalValLoss / valSteps
			# calculate the training and validation accuracy
			trainCorrect = trainCorrect / len(trainDS)
			valCorrect = valCorrect / len(testDS)
			# update our training history
			H["total_train_loss"].append(avgTrainLoss.cpu().detach().numpy())
			H["train_class_acc"].append(trainCorrect)
			H["total_val_loss"].append(avgValLoss.cpu().detach().numpy())
			H["val_class_acc"].append(valCorrect)

  0%|          | 0/20 [00:00<?, ?it/s]


NameError: ignored

**Train loss VS Bouding box loss**<br>
- Train loss: the value of the objective function that you are minimizing.<br>
- Bounding box loss: a loss that measures how "tight" the predicted bounding boxes are to the ground truth object.<br>


**.detach()**<br>
Return a new Tensor, detached from the current grpah. The result will never require gradient. That is, return new tensor from isolated graph<br>
**.cpu()**<br>
Returns a copy of this object stored in CPU memory<br>
**.cpu().detach().numpy()**<br>
To convert tensor in GPU into numpy, move into CPU memory


In [ ]:
# serialize the model to disk
torch.save(objectDetector, Model_Path)
# serialize the label encoder to disk
f = open(Le_Path, "wb")
f.write(pickle.dumps(le))
f.close()
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(H["total_train_loss"], label="total_train_loss")
plt.plot(H["total_val_loss"], label="total_val_loss")
plt.plot(H["train_class_acc"], label="train_class_acc")
plt.plot(H["val_class_acc"], label="val_class_acc")
plt.title("Total Training Loss and Classification Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
# save the training plot
plotPath = os.path.sep.join([Plot_Path, "training.png"])
plt.savefig(plotPath)

# Assessing the object detection training

## Drawing Inference from the Object Detector: predict.py script

In [ ]:
# import the necessary packages
from torchvision import transforms
import mimetypes
import argparse # to write user-friendly commad line interface command.
# argparse = argument parser
import imutils
import pickle
import torch
import cv2
# construct the argument parser and parse the arguments to help the user select the test image
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", required=True,\
help="path to input image/text file of image paths")
args = vars(ap.parse_args()) # return dictionary of given object

**Argument parser**<br>
Argument parser is a kind of communication between the writer of a program and the user. By using argument parser (argparse), the writer provide varables at runtime for the user of its program.<br>
**pickle**<br>
Store python object itself formet not text formet.

In [ ]:
filetype = mimetypes.guess_type(args["input"])[0] # determine the input file type
imagePaths = [args["input"]] # assume that we're working with single input image.
# if the file type is a text file, then we need to process *multiple* images
if "text/plain" == filetype:
  # load the image paths in our testing file by using open()
  imagePaths = open(args["input"]).read().strip().split("\n")
  model = torch.load(Model_Path).to(Device)# load our object detector to call model
  model.eval() # set it evaluation mode
  le = pickle.loads(open(Le_Path, "rb").read()) # load label encoder
                                                # from aforementioned scrip (le = label encoder)

  # define normalization transforms
  transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=Mean, std=STD)
  ])

In [ ]:
# Preprocessing image files to be plugged into the object detector
for imagePath in imagePaths: # take out all image Paths one by one
  image = cv2.imread(imagePath) # load image file in the path
  orig = image.copy() # store before managed image
  # bring channel dimension forward
  # some manage given image file to treat
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # change channel
  image = cv2.resize(image, (224, 224)) # resizing
  image = image.cv2.transpose((2, 0, 1)) # transpose
  image = torch.form_numpy(image)
  image = transforms(image).to(Device) # flash it to the current device
  image = image.unsqueeze(0) # add a batch dimension

  (boxPreds, labelPreds) = model(image) # take out output (prediction)
                                        # from model
  (startX, startY, endX, endY) = boxPreds[0] # take out each element
                                            # from bodxPrediction (unpacking)

  labelPreds = torch.nn.Softmax(dim=1)(labelPreds) # find class label
                              # with higher probability by .softmax()
  i = labelPreds.argmax(dim=-1).cpu() # isloate the index through .argmax()
  label = le.inverse_transform(i)[0] # get name of label as pluging in the le
                                      # and using .inverse_transform

  orig = imutils.resize(orig, width=600) # to fit our screen,
                                        # original image is resized
  (h, w) = orig.shape[:2] # sotre resized image's hight and width
                          # which are used to scale the predicted bbox value
  # Below step is scaling the predicted bbox based on above w, h information
  startX = int(startX * w)
  startY = int(startY * h)
  endX = int(endX * w)
  endY = int(endY * h)

  # To display the bounding bodx's label top of the image
  # below process is doing
  y = startY - 10 if startY - 10 > 10 else startY + 10 # to show on the top
                                                    # we treat Y axis
  cv2.putText(orig, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2) # show text
  cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2) # show rectangle box
                                          # based on bounding box informaitoin

  cv2.imshow("output", orig)
  cv2.waitKey(0)

**Why do we rescale the predicted bbox?**<br>
This is done because we had scaled down the annotations to the range 0 and 1 before fitting them to the model. Hence, all outputs would have to be scaled up for display purposes.

**cv2.waitkey()**<br>
allows you to wait for a specific time in milliseconds until you press any button on the keyword.